In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time  # Automatically downloads the driver



# Automatically download and set up the ChromeDriver
driver = webdriver.Chrome() 
links=[]
for i in range(0,191,20):
    # URL to scrape
    url = f"https://www.reuters.com/site-search/?query=multiple+sclerosis&offset={i}"

    print(url)

https://www.reuters.com/site-search/?query=multiple+sclerosis&offset=0
https://www.reuters.com/site-search/?query=multiple+sclerosis&offset=20
https://www.reuters.com/site-search/?query=multiple+sclerosis&offset=40
https://www.reuters.com/site-search/?query=multiple+sclerosis&offset=60
https://www.reuters.com/site-search/?query=multiple+sclerosis&offset=80
https://www.reuters.com/site-search/?query=multiple+sclerosis&offset=100
https://www.reuters.com/site-search/?query=multiple+sclerosis&offset=120
https://www.reuters.com/site-search/?query=multiple+sclerosis&offset=140
https://www.reuters.com/site-search/?query=multiple+sclerosis&offset=160
https://www.reuters.com/site-search/?query=multiple+sclerosis&offset=180


In [37]:
driver = webdriver.Chrome()

# URL to scrape
url = f"https://www.reuters.com/site-search/?query=multiple+sclerosis&offset=180"

# Open the page
driver.get(url)

# Wait for content to load
driver.implicitly_wait(5)


# Extract links with data-testid="TitleLink"
link_elements = driver.find_elements(By.XPATH, '//a[@data-testid="TitleLink"]')
# Print the extracted links
for element in link_elements:
    
    href = element.get_attribute("href")
    links.append(href)
    print(f"Extracted link: {href}")



Extracted link: https://www.reuters.com/business/merck-kgaa-raises-outlook-after-forecast-beating-q2-profit-2021-08-05/
Extracted link: https://www.reuters.com/legal/government/us-judge-rules-against-perrigo-over-19-billion-irish-tax-liability-2021-07-16/
Extracted link: https://www.reuters.com/legal/transactional/insurance-cos-vp-can-be-deposed-ada-bias-case-11th-circuit-2021-06-21/
Extracted link: https://www.reuters.com/world/the-great-reboot/former-swiss-magic-mountain-tb-clinics-treat-covid-stay-relevant-2021-06-17/
Extracted link: https://www.reuters.com/legal/litigation/5th-circ-revives-would-be-jurors-suit-over-inaccessible-courthouse-2021-06-16/
Extracted link: https://www.reuters.com/business/healthcare-pharmaceuticals/us-fda-set-rule-controversial-biogen-alzheimers-drug-2021-06-07/
Extracted link: https://www.reuters.com/business/healthcare-pharmaceuticals/lawsuit-says-bristol-myers-avoided-64-bln-payment-by-delaying-cancer-drug-2021-06-03/
Extracted link: https://www.reuter

In [38]:
len(links)

190

In [41]:
import csv
with open('href_urls.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["URL"])  # Adding a header
    for url in links:
        writer.writerow([url])

In [40]:
links

['https://www.reuters.com/business/healthcare-pharmaceuticals/fda-warns-serious-allergic-reactions-with-multiple-sclerosis-drugs-2025-01-22/',
 'https://www.reuters.com/legal/novartis-must-face-claims-it-paid-kickbacks-promote-ms-drug-us-appeals-court-2024-12-27/',
 'https://www.reuters.com/business/healthcare-pharmaceuticals/youve-got-mail-medicare-advantage-so-be-careful-2024-11-21/',
 'https://www.reuters.com/business/healthcare-pharmaceuticals/merck-kgaa-q3-earnings-rise-faster-than-expected-lower-costs-2024-11-14/',
 'https://www.reuters.com/business/healthcare-pharmaceuticals/teva-pharmaceutical-third-quarter-profit-revenue-top-estimates-2024-11-06/',
 'https://www.reuters.com/business/healthcare-pharmaceuticals/teva-fined-503-mln-by-eu-disparaging-rival-product-2024-10-31/',
 'https://www.reuters.com/business/healthcare-pharmaceuticals/biogen-lifts-annual-profit-forecast-cost-cuts-help-2024-10-30/',
 'https://www.reuters.com/world/us/tootsie-young-frankenstein-actor-teri-garr-di

In [94]:
import csv
from bs4 import BeautifulSoup
import requests

# Headers for the HTTP request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}



# Open a CSV file to save the data
with open('final.csv', 'w', newline='', encoding='utf-8') as csvfile:
    # Define the CSV writer
    csvwriter = csv.writer(csvfile)
    
    # Write the header row
    csvwriter.writerow(['URL', 'Heading', 'Author', 'Date', 'Content', 'Reported By', 'Edited By'])
    
    for link in links:
        try:
            # Send an HTTP GET request to the link
            html = requests.get(link, headers=headers)
            soup = BeautifulSoup(html.text, 'lxml')
            
            # Extract and handle each field
            heading = soup.find('h1', attrs={'data-testid': 'Heading'})
            heading_text = heading.text.strip() if heading else ""
            
            author = soup.find('div', attrs={'data-testid': 'Text'})
            author_text = author.text.strip().split()[1] if author and len(author.text.strip().split()) > 1 else ""
            
            published_date = soup.find('time', attrs={'data-testid': 'Body'})
            date_spans = published_date.find_all('span', class_='date-line__date___kNbY') if published_date else []
            date_text = date_spans[0].text.strip() if len(date_spans) > 0 else ""
            
            content = soup.find('div', class_='article-body__content__17Yit')
            content_text = content.text.strip() if content else ""
            
            reported_by = soup.find('p', attrs={'data-testid': 'Body'})
            reported_by_text = ""
            edited_by_text = ""
            
            if reported_by:
                body_text = reported_by.text.strip()
                if ';' in body_text:
                    # Split by ';'
                    parts = body_text.split(';')
                    reported_by_text = parts[0].strip() if len(parts) > 0 else ""
                    edited_by_text = parts[1].strip() if len(parts) > 1 else ""
                else:
                    # Handle newlines
                    lines = body_text.splitlines()
                    reported_by_text = lines[0].strip() if len(lines) > 0 else ""
                    edited_by_text = lines[1].strip() if len(lines) > 1 else ""
            
            # Write the row of data to the CSV file
            csvwriter.writerow([link, heading_text, author_text, date_text, content_text, reported_by_text, edited_by_text])
        
        except Exception as e:
            print(f"Error processing link {link}: {e}")
            # Write an empty row in case of an error
            csvwriter.writerow([link, "", "", "", "", "", ""])


In [77]:
import csv

# Initialize an empty list to store URLs
links = []

# Read the CSV file containing the URLs
with open('href_urls.csv', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    
    # Skip the header if it exists (optional)
    next(csvreader)
    
    # Read each row and extract the URL (assuming URL is in the first column)
    for row in csvreader:
        url = row[0]  # Adjust the index if URL is in a different column
        links.append(url)



'https://www.reuters.com/technology/exclusive-swiss-digital-therapeutics-firm-mindmaze-spac-talks-seeks-funds-say-2021-05-05/'

In [91]:
s='''ajd dsFDODF    
DS DSJNV'''.split('\n')
s[1]

'DS DSJNV'